In [1]:
# import libraries
from pyspark.sql import SparkSession

In [2]:
# create a spark session
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

# access the Spark UI by navigating to localhost:4040

### Join two large tables

In [3]:
# read green and yellow revenue report parquet files
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [4]:
# add the service type as a prefix to 'amount' and 'number_records' fields in both dataframes
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [5]:
# join both the dataframes on 'hour' and 'zone' fields
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [6]:
# display the joined dataframe
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|  36|295.34000000000003|                  11|            109.17|                    3|
|2020-01-01 00:00:00|  43|107.52000000000001|                   6|  6539.51000000001|                  390|
|2020-01-01 00:00:00|  68|              NULL|                NULL| 7825.070000000012|                  396|
|2020-01-01 00:00:00|  83|              94.1|                   7|               9.8|                    1|
|2020-01-01 00:00:00| 145|             65.14|                   4|            365.87|                   18|
|2020-01-01 00:00:00| 148|              NULL|                NULL| 6790.150000000003|                  371|
|2020-01-01 00:00:00| 159|  

In [7]:
# save the joined dataset as parquet file
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

### Join a large table and a small table

In [8]:
# read joined revenue report
df_join = spark.read.parquet('data/report/revenue/total')

In [9]:
# display the schema of the dataframe
df_join

DataFrame[hour: timestamp, zone: int, green_amount: double, green_number_records: bigint, yellow_amount: double, yellow_number_records: bigint]

In [10]:
# read taxi zones data
df_zones = spark.read.parquet('data/zones/')

In [11]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [12]:
# join both the dataframes on Zone ID
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [13]:
# display the joined dataframe
df_result.drop('LocationID', 'zone').show()

+-------------------+------------------+--------------------+------------------+---------------------+---------+------------+
|               hour|      green_amount|green_number_records|     yellow_amount|yellow_number_records|  Borough|service_zone|
+-------------------+------------------+--------------------+------------------+---------------------+---------+------------+
|2020-01-01 00:00:00| 769.7299999999997|                  45| 455.1700000000001|                   38|   Queens|   Boro Zone|
|2020-01-01 00:00:00|              NULL|                NULL|             42.41|                    2|   Queens|   Boro Zone|
|2020-01-01 00:00:00|              NULL|                NULL|              19.3|                    1| Brooklyn|   Boro Zone|
|2020-01-01 00:00:00|175.67000000000002|                   6|161.60999999999999|                    7| Brooklyn|   Boro Zone|
|2020-01-01 00:00:00| 98.78999999999999|                   2|              NULL|                 NULL|   Queens|   Bor

In [14]:
# save the joined dataset as parquet file
df_result.drop('LocationID', 'zone').write.parquet('data/tmp/revenue-zones', mode='overwrite')